In [37]:
''#coding: utf-8
# citations: https://pythonmana.com/2021/03/20210329161147051K.html
# citation: https://stackoverflow.com/questions/43637211/retrieve-document-content-with-document-structure-with-python-docx

# Import Libraries
!pip install docx
from docx import Document
from docx.document import Document as _Document
from docx.oxml.text.paragraph import CT_P
from docx.oxml.table import CT_Tbl
from docx.table import _Cell, Table
from docx.text.paragraph import Paragraph
import pandas as pd
import numpy as np
import re
from os.path import exists
!pip install xlrd

In [38]:
########################################################################
# USER INPUTS
########################################################################
# Replace File Path. This is where all the relevant files should be saved.
path = 'C:/Users/jstockham/OneDrive - American Institutes for Research in the Behavioral Sciences/JSmith/PYTHON_CODE_2022/table_automate_jess/general_table_automate/'

######################
# Table Shells in Word
######################

# Word Document Name (If your Word doc is tableshells.docx, report_doc = 'tableshell.docx')
report_doc = 'Reportshell_messy crossref.docx'

# Table Caption Heading Style in Word. Replace the default if needed.
# Note that you cannot have embedded cross-references in the captions
caption_style = 'Exhibit Title'

# Table Style in Word
table_style = '__Table Style-AIR 2021'

# Row Heading Styles in Word. Replace the default if needed.
row_heading_style = 'Table 11 Row Heading'

# Column Heading Styles in Word. Replace the default if needed.
col_heading_style = 'Table 11 Column Heading'

# # Overwrite the header rows of your table shell in the Word doc? Enter lowercase 'y' for yes and 'n' for no.
# overwrite_table_row = 'n'

# # Overwrite the header columns of your table shell in the Word doc? Enter 'y' for yes and 'n' for no.
# overwrite_table_col = 'n'

# Name of updated Word Document - OPTIONAL.  (If your Word doc is outputfile.docx, output_doc = 'outputfile.docx')
# If not provided, will replace the contents of the inputted Word doc
output_doc = ''

######################
# Excel Input Data
######################

# Are all your data tables in a single Excel workbook (1 table per worksheet?) Enter 'y' for yes and 'n' for no.
# If you are pulling data from multiple workbooks, populate that information in the Crosswalk file
single_workbook = 'y'

# Excel Document Name (If your Excel doc is rawdata.xlsx, excel_doc = 'rawdata.xlsx')
excel_doc = 'rawdata1_mess.xlsx'

######################
# Crosswalk File
######################

# Crosswalk Excel Document Name (If your Excel doc is crosswalk.xlsx, crosswalk_doc = 'crosswalk.xlsx')
crosswalk_doc = 'crosswalk_messy.xlsx' 

In [39]:
########################################################################
# Load files
########################################################################

# 1. Load Crosswalk data into Pandas Dataframe
crosswalk_data = pd.read_excel(crosswalk_doc, sheet_name="table settings")
#print(crosswalk_data.head())

# # 2. Load in all sheets from Excel Workbook(s) into Pandas Dataframe
# tables_data = pd.read_excel(excel_doc, sheet_name=None, header=None)
# print(tables_data["Table 1"])

# 3. Load Word Document into a docx file object
document = Document(report_doc)

#4. Set output to be the same as the input Word file
#print(report_doc)
if not output_doc:
    output_doc = report_doc

In [50]:
########################################################################
# Populate Table Data in Word
########################################################################

# Loop through the paragraphs & table pairs in the Word document
#citation: source: https://theprogrammingexpert.com/write-table-fast-python-docx/
''' Define script to identify table 'child' within paragraph 'parent' based on document order
    Each returned value is an instance of either Table or Paragraph. '''
def iter_block_items(parent):
    if isinstance(parent, _Document):
        parent_elm = parent.element.body
    elif isinstance(parent, _Cell):
        parent_elm = parent._tc
    else:
        raise ValueError("something's not right")

    for child in parent_elm.iterchildren():
        if isinstance(child, CT_P):
            yield Paragraph(child, parent)
        elif isinstance(child, CT_Tbl):
            yield Table(child, parent)      

'''Identifies table meta-data from crosswalk file'''
def findtables(document, crosswalk_data):

    # Iterate through paragraphs and table objects in document
    for block in iter_block_items(document):

        # If we encounter a paragraph with the AIR Exhibit Title Formatting
        if isinstance(block, Paragraph):

            # Check if it has Exhibit Title Formatting
            if block.style.name==caption_style:
                print(block.text)

                # Find Exhibit Title in the Crosswalk
                exhibit_name = block.text
#                 block.text.find(substring) 
                      
                # Excel Workbook and Worksheet
                # Need to update this to be .loc rather than .iloc so not sensitive to column ordering
                excel_sheet_name = crosswalk_data.loc[crosswalk_data['word_table_title'] == exhibit_name].iloc[0, 1]
                workbook_name = crosswalk_data.loc[crosswalk_data['word_table_title'] == exhibit_name].iloc[0, 2]
                tables_data = pd.read_excel(workbook_name, sheet_name=excel_sheet_name, header=None)
                #print(f"workbook: {workbook_name}")
                                
#                 # Overwrite Settings
#                 overwrite_table_row = crosswalk_data.loc[crosswalk_data['word_table_title'] == exhibit_name].iloc[0, 3]
#                 overwrite_table_col = crosswalk_data.loc[crosswalk_data['word_table_title'] == exhibit_name].iloc[0, 4]

                # Skip Row and Column Settings
                skip_table_rows = crosswalk_data.loc[crosswalk_data['word_table_title'] == exhibit_name].iloc[0, 3]
                skip_table_cols = crosswalk_data.loc[crosswalk_data['word_table_title'] == exhibit_name].iloc[0, 4]
            
                # Skip Merged Rows
                skip_merged_rows = crosswalk_data.loc[crosswalk_data['word_table_title'] == exhibit_name].iloc[0, 5]
                
                # table=document.tables[0]
                # row = table.rows[0]
                # len(row.cells)
                # len(Column.cells

                # c=table.cell(0,0)
                # print(c.text, c._tc.top, c._tc.bottom, c._tc.left, c._tc.right)

                # c=table.cell(1,0)
                # print(c.text, c._tc.top, c._tc.bottom, c._tc.left, c._tc.right)

                # c=table.cell(2,0)
                # print(c.text, c._tc.top, c._tc.bottom, c._tc.left, c._tc.right)

                # Continue to the next block item, which is the corresponding table
                continue

        # Access the table object
        elif isinstance(block,Table):
            
            tablepopulate(block, tables_data, skip_table_rows, skip_table_cols, skip_merged_rows)

'''Extracts that table's formatting specifications and fills in the data'''
def tablepopulate(block, df, skip_table_rows, skip_table_cols, skip_merged_rows):
        print(df.head())
        # loop through rows and cols of the dataframe to populate the table object
        for i in range(skip_table_rows, df.shape[0]):
            for j in range(skip_table_cols, df.shape[1]):
                print(j)
                # Skip merged cells in the first column
                if j == 0 and skip_merged_rows == 'y':
                    c=block.cell(i,0)
                    print(c.text, c._tc.top, c._tc.bottom, c._tc.left, c._tc.right)
                    if c._tc.right > 1:
                        print(j)
                        continue
                                      
                # Skip over blank (nan) cells in the dataframe
                if (str(df.values[i,j])) != 'nan':
                    #print(str(df.values[i,j]))
                    block.cell(i,j).text = str(df.values[i,j])
        
        # Add table styles and formats                        
        block.style = table_style

findtables(document, crosswalk_data)

#3. Save the outputted Word document
#document.save(output_doc)

document.save('result.docx')

Exhibit 1. I am first
          0      1      2      3
0  Response    Yes     No  Maybe
1     Total  0.028  0.012  0.026
2   Group A    NaN    NaN    NaN
3    Site 1  0.095  0.106  0.097
4    Site 2  0.004      0  0.004
0
Response 0 1 0 1
1
2
3
0
Total 1 2 0 1
1
2
3
0
nan 2 3 0 4
0
1
2
3
0
Site 1 3 4 0 1
1
2
3
0
Site 2 4 5 0 1
1
2
3
0
nan 5 6 0 4
0
1
2
3
0
Site 1 6 7 0 1
1
2
3
0
Site 2 7 8 0 1
1
2
3
Exhibit 2. I am second
                    0                 1       2       3       4       5
0              Status           Outcome  Site 1  Site 2  Site 3  Site 4
1  Education Outcomes  HS Degree Earned    0.41    0.37    0.41    0.37
2                 NaN        GED Earned    0.59    0.63    0.59    0.63
3    Program Outcomes            Exiter    0.41    0.37    0.41    0.37
4                 NaN         Completer    0.59    0.63    0.59    0.63
0
Status 0 1 0 1
1
2
3
4
5
0
Education Outcomes 1 3 0 1
1
2
3
4
5
0
Education Outcomes 1 3 0 1
1
2
3
4
5
0
Program Outcomes 3 5 0 1
1
2
3
4
5


In [ ]:
                #print(f": i {i} j {j}")
                #print(pd.isna(df[i][j]))
                
#                 #Check for merged cells in the first column
#                 if j == 0:
#                     c=table.cell(i,0)
#                     print(c._tc.right)
#                     print(c.text)
#                     if c._tc.right > 1:
#                         continue
#                     block.cell(i,j).text = str(df.values[i,j])
                
#                 else:
#                     block.cell(i,j).text = str(df.values[i,j])
                    
#                     # c=table.cell(1,0)
# # print(c.text, c._tc.top, c._tc.bottom, c._tc.left, c._tc.right)
                                       

In [ ]:

#             except:

#                 print("f({exhibit_name} does not exist in the Excel file."
#                       f"Please double-check your Excel file information for typos")


#             print(f"first column {block.row.cells[0].text}")
#             print(f"first row {block.columns.cells[0].text}")

#             row = block.rows[0]
#             # Access the first column of the row
#             row.cells[0].text
#             # Access the second column of the row
#             row.cells[1].text = 'This is the second row, second column'

#             block.rows[0].style = col_heading_style  # first column heading style
#             block.columns[0].style = row_heading_style   # first row heading style
              
              #                 value = df.at[i, j] #get cell value
#                 print(np.isnan(value), end="\t")
#                 if pd.isna(df[i][j]):                    
#                     print(df.values[i,j])

In [ ]:
# table=document.tables[0]
# row = table.rows[0]
# len(row.cells)
# len(Column.cells

# c=table.cell(0,0)
# print(c.text, c._tc.top, c._tc.bottom, c._tc.left, c._tc.right)

# c=table.cell(1,0)
# print(c.text, c._tc.top, c._tc.bottom, c._tc.left, c._tc.right)

# c=table.cell(2,0)
# print(c.text, c._tc.top, c._tc.bottom, c._tc.left, c._tc.right)

# c=table.cell(3,0)
# print(c.text, c._tc.top, c._tc.bottom, c._tc.left, c._tc.right)

# c=table.cell(4,0)
# print(c.text, c._tc.top, c._tc.bottom, c._tc.left, c._tc.right)

# c=table.cell(5,0)
# print(c.text, c._tc.top, c._tc.bottom, c._tc.left, c._tc.right)




# c=table.cell(0,1)
# print(c.text, c._tc.top, c._tc.bottom, c._tc.left, c._tc.right)

# c=table.cell(1,1)
# print(c.text, c._tc.top, c._tc.bottom, c._tc.left, c._tc.right)

# c=table.cell(2,1)
# print(c.text, c._tc.top, c._tc.bottom, c._tc.left, c._tc.right)

# c=table.cell(3,1)
# print(c.text, c._tc.top, c._tc.bottom, c._tc.left, c._tc.right)

# c=table.cell(4,1)
# print(c.text, c._tc.top, c._tc.bottom, c._tc.left, c._tc.right)

# c=table.cell(5,1)
# print(c.text, c._tc.top, c._tc.bottom, c._tc.left, c._tc.right)

In [ ]:
        # Set the initial row for iteration in the dataframe (default = 0)
        initial_row = 0
        if overwrite_table_row == 'n': 
            initial_row = 1 
        print(f"initial_row {initial_row}")

        # Set the initial col for iteration in the dataframe (default = 0)
        initial_col = 0
        if overwrite_table_col == 'n': 
            initial_col = 1
        print(f"initial_col {initial_col}")